In [26]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from read_tab_files import TabFileReader

In [33]:
barb_forms = TabFileReader.tab_reader(
    "chl2024_barbacoandata/chl2023_barbacoan_forms.tab"
)

word_list = TabFileReader.get_word_list(barb_forms)

# convert words to character sequences
tokens = [list(word) for word in word_list]

# flatten the list of chars to fit LabelEncoder
flat_tokens = [item for sublist in tokens for item in sublist]

# encode characters as integers
encoder = LabelEncoder()
encoder.fit(flat_tokens)
encoded_tokens = [encoder.transform(word) for word in tokens]

# specify sequence length (how long each seq should be interpreted as)
sequence_length = 4

X = []
y = []

# go through each word and get every sequence with the seq len (hello > hel, ell, llo if seq_len = 3)
for seq in encoded_tokens:
    for i in range(sequence_length, len(seq)):
        X.append(seq[i - sequence_length : i])
        y.append(seq[i])


# prep data for training
X = np.array(X)
# reshape X to fit the LSTM input [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
y = to_categorical(y)

In [34]:
model = Sequential(
    [
        # memory units (50 is good starting num), seq_len, step size (1)
        LSTM(100, input_shape=(X.shape[1], X.shape[2])),
        Dropout(0.2),
        # num unique chars
        Dense(y.shape[1], activation="softmax"),
    ]
)

model.compile(loss="categorical_crossentropy", optimizer="adam")
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
39/39 [==============================] - 1s 2ms/step - loss: 3.0907
Epoch 2/200
39/39 [==============================] - 0s 2ms/step - loss: 2.7638
Epoch 3/200
39/39 [==============================] - 0s 2ms/step - loss: 2.7260
Epoch 4/200
39/39 [==============================] - 0s 2ms/step - loss: 2.7031
Epoch 5/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6892
Epoch 6/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6693
Epoch 7/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6640
Epoch 8/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6291
Epoch 9/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6243
Epoch 10/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6072
Epoch 11/200
39/39 [==============================] - 0s 2ms/step - loss: 2.6058
Epoch 12/200
39/39 [==============================] - 0s 2ms/step - loss: 2.5929
Epoch 13/200
39/39 [=================

In [35]:
def generate_word(model, input_text, extend_length, encoder):
    # pad text if its length is less than seq_len
    if len(input_text) < sequence_length:
        input_text = " " * (sequence_length - len(input_text)) + input_text

    for _ in range(extend_length):
        encoded = encoder.transform(list(input_text[-sequence_length:]))
        encoded = np.reshape(encoded, (1, sequence_length, 1))

        # predict next character
        pred = model.predict(encoded, verbose=0)
        next_char = encoder.inverse_transform([np.argmax(pred)])
        input_text += next_char[0]

    return input_text

In [38]:
cognates = ["pi", "chu", "mɨ", "lul"]
lengths = list(range(1, 6))

for cognate in cognates:
    print("Cognate:", cognate)
    for length in lengths:
        new_word = generate_word(model, cognate, length, encoder)
        print("  Extra Length:", length)
        print("  Generated Word:", new_word)
    print()

Cognate: pi
  Extra Length: 1
  Generated Word:   pin
  Extra Length: 2
  Generated Word:   pina
  Extra Length: 3
  Generated Word:   pinan
  Extra Length: 4
  Generated Word:   pinanp
  Extra Length: 5
  Generated Word:   pinanpa

Cognate: chu
  Extra Length: 1
  Generated Word:  chun
  Extra Length: 2
  Generated Word:  chunk
  Extra Length: 3
  Generated Word:  chunka
  Extra Length: 4
  Generated Word:  chunka 
  Extra Length: 5
  Generated Word:  chunka m

Cognate: mɨ
  Extra Length: 1
  Generated Word:   mɨp
  Extra Length: 2
  Generated Word:   mɨpɨ
  Extra Length: 3
  Generated Word:   mɨpɨp
  Extra Length: 4
  Generated Word:   mɨpɨpɨ
  Extra Length: 5
  Generated Word:   mɨpɨpɨp

Cognate: lul
  Extra Length: 1
  Generated Word:  lulɨ
  Extra Length: 2
  Generated Word:  lulɨp
  Extra Length: 3
  Generated Word:  lulɨpɨ
  Extra Length: 4
  Generated Word:  lulɨpɨp
  Extra Length: 5
  Generated Word:  lulɨpɨpɨ



In [31]:
new_word = generate_word(model, "pi", 2, encoder)
print("Generated word:", new_word)

Generated word: pika
Generated word: chuhka
Generated word: mɨm
Generated word: lulan
